# Benchmark of various outlier detection model thresholders

### The models are evaluated by ROC, Precision @ n and execution time on 17 benchmark datasets. All datasets are split (60% for training and 40% for testing). 

The thresholders covered in this example include:

1. **IQR: Inter-Quartile Region** 
2. **MAD: Median Absolute Deviation**
3. **FWFM: Full Width at Full Minimum**
4. **YJ: Yeo-Johnson Transformation**
5. **ZSCORE: Z Score**
6. **AUCP: Area Under the Curve Percentage**
7. **QMCD: Quasi-Monte Carlo Discreperancy**
8. **FGD: Fixed Gradient Descent**
9. **DSN: Distance Shift from Normal**
10. **CLF: Trained Classifier**
11. **FILTER: Filtering Based**
12. **WIND: Topological Winding Number**
13. **EB: Elliptical Boundary**
14. **REGR: Regression Intercept**
15. **BOOT: Bootstrap Method**
16. **MCST: Monte Carlo Statistical Tests**
17. **HIST: Histogram Based Methods**
18. **MOLL: Mollifier**
19. **CHAU: Chauvenet's Criterion**
20. **GESD: Generalized Extreme Studentized Deviate**
21. **MTT: Modified Thompson Tau Test**
22. **KARCH: Karcher Mean**
23. **OCSVM: One-Class SVM**
24. **CLUST: Clustering**

In [1]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pythresh is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

import numpy as np
from numpy import percentile
import matplotlib.pyplot as plt
import matplotlib.font_manager
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.knn import KNN
from pyod.models.iforest import IForest

from pythresh.thresholds.iqr import IQR
from pythresh.thresholds.mad import MAD
from pythresh.thresholds.fwfm import FWFM
from pythresh.thresholds.yj import YJ
from pythresh.thresholds.zscore import ZSCORE
from pythresh.thresholds.aucp import AUCP
from pythresh.thresholds.qmcd import QMCD
from pythresh.thresholds.fgd import FGD
from pythresh.thresholds.dsn import DSN
from pythresh.thresholds.clf import CLF
from pythresh.thresholds.filter import FILTER
from pythresh.thresholds.wind import WIND
from pythresh.thresholds.eb import EB
from pythresh.thresholds.regr import REGR
from pythresh.thresholds.boot import BOOT
from pythresh.thresholds.mcst import MCST
from pythresh.thresholds.hist import HIST
from pythresh.thresholds.moll import MOLL
from pythresh.thresholds.chau import CHAU
from pythresh.thresholds.gesd import GESD
from pythresh.thresholds.mtt import MTT
from pythresh.thresholds.karch import KARCH
from pythresh.thresholds.ocsvm import OCSVM
from pythresh.thresholds.clust import CLUST

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

INFO: Using numpy backend


In [3]:
# Define data file and read X and y
mat_file_list = ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat',
                 'ionosphere.mat',
                 'letter.mat',
                 'lympho.mat',
                 'mnist.mat',
                 'musk.mat',
                 'optdigits.mat',
                 'pendigits.mat',
                 'pima.mat',
                 'satellite.mat',
                 'satimage-2.mat',
                 'vertebral.mat',
                 'vowels.mat',
                 'wbc.mat']

# Define nine outlier detection tools to be compared
random_state = np.random.RandomState(42)

df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc', 'IQR', 'MAD', 'FWFM', 
              'YJ', 'ZSCORE', 'AUCP', 'QMCD', 'FGD', 'DSN', 'CLF', 'FILTER', 'WIND', 
              'EB', 'REGR', 'BOOT', 'MCST', 'HIST', 'MOLL', 'CHAU', 'GESD', 'MTT', 
              'KARCH', 'OCSVM', 'CLUST']

roc_df = pd.DataFrame(columns=df_columns)
prn_df = pd.DataFrame(columns=df_columns)
time_df = pd.DataFrame(columns=df_columns)

clf = IForest()


for mat_file in mat_file_list:
    print("\n... Processing", mat_file, '...')
    mat = loadmat(os.path.join('data', mat_file))

    X = mat['X']
    y = mat['y'].ravel()
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)

    # construct containers for saving results
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

    # 60% data for training and 40% for testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                        random_state=random_state)

    # standardizing data for processing
    X_train_norm, X_test_norm = standardizer(X_train, X_test)

    thresholders = {
        'Inter-Quartile Region (IQR)':IQR(),
        'Median Absolute Deviation (MAD)':MAD(),
        'Full Width at Full Minimum (FWFM)':FWFM(),
        'Yeo-Johnson Transformation (YJ)': YJ(),
        'Z Score (ZSCORE)': ZSCORE(),
        'AUC Percentage (AUCP)': AUCP(),
        'Quasi-Monte Carlo Discreperancy (QMCD)': QMCD(),
        'Fixed Gradient Descent (FGD)': FGD(),
        'Distance Shift from Normal (DSN)': DSN(),
        'Trained Classifier (CLF)': CLF(),
        'Filtering Based (FILTER)': FILTER(),
        'Topological Winding Number (WIND)': WIND(),
        'Elliptical Boundary (EB)': EB(),
        'Regression Intercept (REGR)': REGR(),
        'Bootstrap Method (BOOT)': BOOT(),
        'Monte Carlo Statistical Tests (MCST)': MCST(),
        'Histogram Based Methods (HIST)': HIST(),
        'Mollifier (MOLL)': MOLL(),
        "Chauvenet's Criterion (CHAU)": CHAU(),
        'Generalized Extreme Studentized Deviate (GESD)': GESD(),
        'Modified Thompson Tau Test (MTT)': MTT(),
        'Karcher Mean (KARCH)': KARCH(),
        'One-Class SVM (OCSVM)': OCSVM(),
        'Clustering (CLUST)': CLUST()
    }
    
    clf.fit(X_train_norm)
    scores = clf.decision_scores_
    
    for thres_name, thres in thresholders.items():
        t0 = time()
        pred = thres.eval(scores)
        contam = np.sum(pred)/len(pred)
        
        if contam<=0: contam=1e-3
        if contam>0.5: contam=0.5
        
        clf = IForest(contamination=contam)
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1 - t0, ndigits=4)
        time_list.append(duration)

        roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

        print('{thres_name} ROC:{roc}, precision @ rank n:{prn}, '
              'execution time: {duration}s'.format(
        thres_name=thres_name, roc=roc, prn=prn, duration=duration))

        roc_list.append(roc)
        prn_list.append(prn)

    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)

    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)

    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)


... Processing arrhythmia.mat ...
Inter-Quartile Region (IQR) ROC:0.8382, precision @ rank n:0.5, execution time: 0.7416s
Median Absolute Deviation (MAD) ROC:0.8275, precision @ rank n:0.5357, execution time: 0.7379s
Full Width at Full Minimum (FWFM) ROC:0.8224, precision @ rank n:0.5714, execution time: 0.8018s
Yeo-Johnson Transformation (YJ) ROC:0.8522, precision @ rank n:0.5357, execution time: 0.9908s
Z Score (ZSCORE) ROC:0.8359, precision @ rank n:0.5357, execution time: 0.7256s
AUC Percentage (AUCP) ROC:0.8315, precision @ rank n:0.5, execution time: 0.7927s
Quasi-Monte Carlo Discreperancy (QMCD) ROC:0.838, precision @ rank n:0.5714, execution time: 0.7562s
Fixed Gradient Descent (FGD) ROC:0.8513, precision @ rank n:0.5357, execution time: 0.8446s
Distance Shift from Normal (DSN) ROC:0.8408, precision @ rank n:0.5357, execution time: 0.752s
Trained Classifier (CLF) ROC:0.8193, precision @ rank n:0.5, execution time: 0.7568s
Filtering Based (FILTER) ROC:0.8238, precision @ rank n

Chauvenet's Criterion (CHAU) ROC:0.7822, precision @ rank n:0.587, execution time: 0.5681s
Generalized Extreme Studentized Deviate (GESD) ROC:0.7918, precision @ rank n:0.5652, execution time: 0.6323s
Modified Thompson Tau Test (MTT) ROC:0.7966, precision @ rank n:0.6304, execution time: 0.6297s
Karcher Mean (KARCH) ROC:0.7991, precision @ rank n:0.5652, execution time: 0.5646s
One-Class SVM (OCSVM) ROC:0.7851, precision @ rank n:0.5652, execution time: 0.7273s
Clustering (CLUST) ROC:0.7876, precision @ rank n:0.5217, execution time: 0.5739s

... Processing letter.mat ...
Inter-Quartile Region (IQR) ROC:0.6003, precision @ rank n:0.0488, execution time: 0.7628s
Median Absolute Deviation (MAD) ROC:0.5881, precision @ rank n:0.0488, execution time: 0.7962s
Full Width at Full Minimum (FWFM) ROC:0.6061, precision @ rank n:0.0488, execution time: 0.8864s
Yeo-Johnson Transformation (YJ) ROC:0.5978, precision @ rank n:0.0976, execution time: 1.4273s
Z Score (ZSCORE) ROC:0.5907, precision @ ra

Elliptical Boundary (EB) ROC:0.9998, precision @ rank n:0.9756, execution time: 2.339s
Regression Intercept (REGR) ROC:1.0, precision @ rank n:1.0, execution time: 1.915s
Bootstrap Method (BOOT) ROC:1.0, precision @ rank n:1.0, execution time: 2.161s
Monte Carlo Statistical Tests (MCST) ROC:0.9994, precision @ rank n:0.9268, execution time: 1.9231s
Histogram Based Methods (HIST) ROC:0.9989, precision @ rank n:0.878, execution time: 1.5734s
Mollifier (MOLL) ROC:0.9977, precision @ rank n:0.875, execution time: 1.5179s
Chauvenet's Criterion (CHAU) ROC:1.0, precision @ rank n:1.0, execution time: 1.5441s
Generalized Extreme Studentized Deviate (GESD) ROC:1.0, precision @ rank n:0.9756, execution time: 2.1576s
Modified Thompson Tau Test (MTT) ROC:1.0, precision @ rank n:1.0, execution time: 1.5319s
Karcher Mean (KARCH) ROC:1.0, precision @ rank n:1.0, execution time: 1.6769s
One-Class SVM (OCSVM) ROC:0.999, precision @ rank n:0.9024, execution time: 2.2978s
Clustering (CLUST) ROC:0.9996, p

AUC Percentage (AUCP) ROC:0.6862, precision @ rank n:0.569, execution time: 2.7928s
Quasi-Monte Carlo Discreperancy (QMCD) ROC:0.7164, precision @ rank n:0.5924, execution time: 1.3713s
Fixed Gradient Descent (FGD) ROC:0.6956, precision @ rank n:0.564, execution time: 2.6923s
Distance Shift from Normal (DSN) ROC:0.6795, precision @ rank n:0.5591, execution time: 3.1861s
Trained Classifier (CLF) ROC:0.7443, precision @ rank n:0.5813, execution time: 1.3225s
Filtering Based (FILTER) ROC:0.6938, precision @ rank n:0.5554, execution time: 1.3394s
Topological Winding Number (WIND) ROC:0.7092, precision @ rank n:0.5603, execution time: 3.2804s
Elliptical Boundary (EB) ROC:0.6837, precision @ rank n:0.5579, execution time: 3.3089s
Regression Intercept (REGR) ROC:0.6785, precision @ rank n:0.548, execution time: 2.5891s
Bootstrap Method (BOOT) ROC:0.6952, precision @ rank n:0.5628, execution time: 2.8285s
Monte Carlo Statistical Tests (MCST) ROC:0.7268, precision @ rank n:0.5825, execution tim

One-Class SVM (OCSVM) ROC:0.8149, precision @ rank n:0.2632, execution time: 0.9828s
Clustering (CLUST) ROC:0.7619, precision @ rank n:0.2105, execution time: 0.736s

... Processing wbc.mat ...
Inter-Quartile Region (IQR) ROC:0.9192, precision @ rank n:0.5556, execution time: 0.5476s
Median Absolute Deviation (MAD) ROC:0.9332, precision @ rank n:0.5556, execution time: 0.5791s
Full Width at Full Minimum (FWFM) ROC:0.934, precision @ rank n:0.5556, execution time: 0.6272s
Yeo-Johnson Transformation (YJ) ROC:0.9223, precision @ rank n:0.5556, execution time: 0.805s
Z Score (ZSCORE) ROC:0.9114, precision @ rank n:0.4444, execution time: 0.5895s
AUC Percentage (AUCP) ROC:0.9409, precision @ rank n:0.5556, execution time: 0.5789s
Quasi-Monte Carlo Discreperancy (QMCD) ROC:0.9262, precision @ rank n:0.4444, execution time: 0.6375s
Fixed Gradient Descent (FGD) ROC:0.9044, precision @ rank n:0.4444, execution time: 0.5761s
Distance Shift from Normal (DSN) ROC:0.9316, precision @ rank n:0.5556,

In [4]:
print('Time complexity')
time_df

Time complexity


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,BOOT,MCST,HIST,MOLL,CHAU,GESD,MTT,KARCH,OCSVM,CLUST
0,arrhythmia,452,274,14.6018,0.7416,0.7379,0.8018,0.9908,0.7256,0.7927,...,0.9253,0.8722,0.7472,0.759,0.7499,1.1451,0.9106,0.8696,1.1271,1.016
0,cardio,1831,21,9.6122,0.853,0.8964,1.0595,2.0566,1.018,0.9547,...,1.3042,0.887,0.7261,0.7171,0.7795,1.0667,0.7508,0.752,1.1071,0.9826
0,glass,214,9,4.2056,0.5168,0.5264,0.5282,0.7521,0.6582,0.5619,...,0.5815,0.5321,0.5268,0.5299,0.525,0.5644,0.5261,0.5301,0.674,0.5294
0,ionosphere,351,33,35.8974,0.6263,0.6015,0.5795,0.8189,0.567,0.6007,...,0.6564,0.5795,0.6081,0.6108,0.5681,0.6323,0.6297,0.5646,0.7273,0.5739
0,letter,1600,32,6.25,0.7628,0.7962,0.8864,1.4273,0.7631,0.9968,...,0.9989,0.8477,0.7142,0.7498,0.7652,1.0407,0.7208,0.8025,1.0856,0.8426
0,lympho,148,18,4.0541,0.5159,0.5221,0.5224,0.7218,0.5587,0.6217,...,0.5546,0.5584,0.5514,0.5238,0.557,0.5952,0.5221,0.5224,0.6997,0.5255
0,mnist,7603,100,9.2069,2.1676,2.1938,4.3532,5.603,2.1835,4.1683,...,5.4539,4.7009,2.1868,2.1972,2.1182,3.6548,2.2316,3.0562,5.4659,6.4292
0,musk,3062,166,3.1679,1.5602,1.575,1.8481,2.4263,1.5677,1.9945,...,2.161,1.9231,1.5734,1.5179,1.5441,2.1576,1.5319,1.6769,2.2978,2.0831
0,optdigits,5216,64,2.8758,1.5066,1.4465,2.4605,3.2139,1.4456,2.591,...,2.8497,2.6549,1.4747,1.4554,1.4556,2.4792,1.5725,1.882,3.2236,3.4116
0,pendigits,6870,16,2.2707,1.2574,1.2157,2.8735,4.2148,1.2657,2.9133,...,3.9111,3.2716,1.2104,1.2746,1.1979,2.5082,1.2259,1.9161,3.7107,5.6477


In [5]:
print('ROC Performance')
roc_df

ROC Performance


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,BOOT,MCST,HIST,MOLL,CHAU,GESD,MTT,KARCH,OCSVM,CLUST
0,arrhythmia,452,274,14.6018,0.8382,0.8275,0.8224,0.8522,0.8359,0.8315,...,0.8504,0.8366,0.8347,0.8606,0.835,0.8217,0.8452,0.8506,0.8473,0.841
0,cardio,1831,21,9.6122,0.9287,0.9214,0.9187,0.9141,0.912,0.9125,...,0.9212,0.9268,0.9321,0.9418,0.9368,0.9196,0.927,0.9226,0.9417,0.9359
0,glass,214,9,4.2056,0.6617,0.6741,0.637,0.6543,0.6296,0.637,...,0.6519,0.6642,0.6765,0.6568,0.6272,0.6296,0.6691,0.6469,0.6494,0.6346
0,ionosphere,351,33,35.8974,0.7817,0.7936,0.77,0.7719,0.7865,0.7828,...,0.7833,0.7803,0.7517,0.7927,0.7822,0.7918,0.7966,0.7991,0.7851,0.7876
0,letter,1600,32,6.25,0.6003,0.5881,0.6061,0.5978,0.5907,0.5804,...,0.6158,0.5971,0.609,0.6056,0.6075,0.621,0.5939,0.627,0.6163,0.6236
0,lympho,148,18,4.0541,1.0,1.0,1.0,0.9942,1.0,1.0,...,1.0,0.9942,1.0,1.0,0.9883,1.0,1.0,1.0,0.9942,1.0
0,mnist,7603,100,9.2069,0.8219,0.7866,0.7966,0.7782,0.8217,0.7758,...,0.782,0.8059,0.7782,0.783,0.8099,0.7943,0.7816,0.8156,0.8182,0.7915
0,musk,3062,166,3.1679,0.9953,0.9997,1.0,0.9972,1.0,1.0,...,1.0,0.9994,0.9989,0.9977,1.0,1.0,1.0,1.0,0.999,0.9996
0,optdigits,5216,64,2.8758,0.7187,0.665,0.7445,0.7259,0.7207,0.7661,...,0.7858,0.7038,0.7277,0.7493,0.7264,0.7756,0.712,0.7614,0.7351,0.7629
0,pendigits,6870,16,2.2707,0.9529,0.9556,0.9338,0.9421,0.9427,0.9207,...,0.9491,0.9497,0.9557,0.9447,0.949,0.9284,0.9578,0.9206,0.9475,0.9429


In [6]:
print('Precision @ n Performance')
prn_df

Precision @ n Performance


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,BOOT,MCST,HIST,MOLL,CHAU,GESD,MTT,KARCH,OCSVM,CLUST
0,arrhythmia,452,274,14.6018,0.5,0.5357,0.5714,0.5357,0.5357,0.5,...,0.6071,0.6071,0.5,0.5,0.5357,0.5357,0.6071,0.5714,0.5357,0.6429
0,cardio,1831,21,9.6122,0.5,0.4857,0.5429,0.4714,0.4429,0.4857,...,0.4857,0.5286,0.5714,0.5714,0.5286,0.4286,0.5,0.5286,0.5,0.5429
0,glass,214,9,4.2056,0.2,0.2,0.2,0.2,0.2,0.2,...,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
0,ionosphere,351,33,35.8974,0.5652,0.5435,0.5217,0.5652,0.5435,0.5652,...,0.5652,0.5652,0.5217,0.587,0.587,0.5652,0.6304,0.5652,0.5652,0.5217
0,letter,1600,32,6.25,0.0488,0.0488,0.0488,0.0976,0.0976,0.0732,...,0.0732,0.0732,0.0732,0.0976,0.0732,0.0732,0.0488,0.0976,0.122,0.0976
0,lympho,148,18,4.0541,1.0,1.0,1.0,0.6667,1.0,1.0,...,1.0,0.6667,1.0,1.0,0.6667,1.0,1.0,1.0,0.6667,1.0
0,mnist,7603,100,9.2069,0.3444,0.263,0.2667,0.2593,0.3556,0.2556,...,0.263,0.2852,0.2667,0.3111,0.3037,0.2519,0.2444,0.2963,0.3222,0.2741
0,musk,3062,166,3.1679,0.8049,0.9512,1.0,0.8293,1.0,1.0,...,1.0,0.9268,0.878,0.875,1.0,0.9756,1.0,1.0,0.9024,0.9512
0,optdigits,5216,64,2.8758,0.0462,0.0462,0.0308,0.0,0.0462,0.0462,...,0.0462,0.0154,0.0462,0.0615,0.0154,0.0462,0.0154,0.0615,0.0462,0.0615
0,pendigits,6870,16,2.2707,0.2581,0.3065,0.2097,0.3065,0.2419,0.2581,...,0.2581,0.2742,0.3871,0.2581,0.2581,0.1613,0.2903,0.2097,0.1935,0.3226
